# [CoE202] **[Homework3]** CNN Classification for CIFAR10 (Pytorch)

## **Testing Script**

In this section, you are going to **test** CNN classification for CIFAR10 in the Pytorch framework.


## 0. Import Library

In [ ]:
import numpy as np
import matplotlib.pyplot as plt # this is for importing matplotlib.pyplot (library for graph plot)
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##1. Define your model to load weights, and define your transfomation in Dataloader

In [ ]:
#You can only change this cell

#define your model
class CIFAR_Net(nn.Module): 
    def __init__(self):
        super(CIFAR_Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(3 * 32 * 32, 200)
        self.fc1 = nn.Linear(200, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32) # x.view convert the shape of tensor, (Batch_size, 3, 32, 32) --> (Batch_size, 3 * 32 * 32)
        x = self.fc0(x) # 3 * 32 * 32 -> 200
        x = torch.sigmoid(x) # Activation function 
        x = self.fc1(x) # 200 --> 50
        x = torch.sigmoid(x) # Activation function 
        x = self.fc2(x)  # 50 -> 10, logit for each class

        return x

#define your transformation on input image, groundtruth
transform = transforms.Compose([
    transforms.ToTensor(),
])
target_transform = None


## 2. Load test dataset

In [ ]:
test_dataset = torchvision.datasets.CIFAR10("../cifar10_data/",
                                              download=True,
                                              train=False,
                                              transform=transform,
                                              target_transform=target_transform)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 100, shuffle=False, drop_last = True)




Extracting ../cifar10_data/cifar-10-python.tar.gz to ../cifar10_data/


##3. Load your model weights

In [ ]:
model_save_name = 'checkpoint.pth'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model = CIFAR_Net()
model.load_state_dict(torch.load(path))
model.cuda()
model.eval()

CIFAR_Net(
  (fc0): Linear(in_features=3072, out_features=200, bias=True)
  (fc1): Linear(in_features=200, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

##4. Check model accuracy

In [ ]:
def thresholding(prediction):
    """Find label which shows highest prediction value.
    
    Args:
        prediction (torch.tensor) : prediction tensor. Dimension of [Batch size, 10]
    
    Returns:
        pred_label (torch.tensor) : Index which shows maximum prediction value. Dimension of [Batch size]
    """
    _, pred_label = torch.max(prediction, 1)
    return pred_label

In [ ]:
model.eval()
total_cnt, correct_cnt = 0.0, 0.0
for batch_idx, (x, target) in enumerate(test_loader):
    with torch.no_grad():
      x, target = x.cuda(), target.cuda()
      prediction = model(x)
      total_cnt += x.data.size(0)
      correct_cnt += (thresholding(prediction) == target.data).sum().item()

print(f"Test Accuracy = {100*correct_cnt/total_cnt}%")

Test Accuracy = 52.56%
